<a href="https://github.com/theonaunheim">
    <img style="border-radius: 100%; float: right;" src="static/strawberry_thief_square.png" width=10% alt="Theo Naunheim's Github">
</a>
<br style="clear: both">
<hr>
<br>


<div style="display: table; width: 100%">
    <div style="display: table-row; width: 100%;">
        <div style="display: table-cell; width: 50%; vertical-align: middle;">
            <img src="static/database.png" width="200">
        </div>
        <div style="display: table-cell; width: 10%">
        </div>
        <div style="display: table-cell; width: 40%; vertical-align: top;">
            <blockquote>
                <p style="font-style: italic;">"Syntax Error Near ' , '."</p>
                <br>
                <p>-Every RDBMS Ever</p>
            </blockquote>
        </div>
    </div>
</div>

<br>

<div align='left'>
    Image courtesy of <a href='https://commons.wikimedia.org/wiki/File:Applications-database.svg'>Dracos</a> under the <a href='https://creativecommons.org/licenses/by-sa/3.0/deed.en'>CC BY-SA 3.0</a>
</div>

<hr>

# Generally

Odds are you're going to need information from a database at some point. Python has got you covered. Here we're going to look through some of the more common libraries for querying, creating, and tinkering with databases.


---

# Modules covered

### Standard Library
* [csv](https://docs.python.org/3/library/csv.html)
* [getpass](https://docs.python.org/3.6/library/getpass.html)
* [os](https://docs.python.org/3/library/os.html)
* [sqlite3](https://docs.python.org/3/library/sqlite3.html)

### Third Party Libraries
* [pandas](https://pandas.pydata.org/pandas-docs/stable/)
* [pyodbc](https://github.com/mkleehammer/pyodbc/wiki)


# Modules not covered

### Standard Library
* None

### Third Party Libraries
* [sqlalchemy](https://www.sqlalchemy.org/)

---

In [1]:
# Stdlib imports
import csv
import getpass
import os
import sqlite3

# Third party imports
import pandas as pd
import pyodbc

# Connecting to databases

### Note: to make this work, you need to have:

a) the appropriate ODBC drivers to access a particular type of database; and,

b) access and permissions for a working database.

The examples below will not work unless you customize them for the particular database you are connecting to. If you want to create your own non-SQLite databases to test on, take a look at the following:

* [SQL Server Dev/SQL Server Express](https://www.microsoft.com/en-us/sql-server/sql-server-downloads)
* [Oracle Express](https://www.oracle.com/database/technologies/appdev/xe.html)

Note: these may have significant technical or licensing limitations that would render them unsuitable for things other than tinkering.

In [2]:
print('{} has the following ODBC drivers available for PyODBC:\n'.format(os.environ['COMPUTERNAME']))
for driver in pyodbc.drivers():
    print(driver)

KILO has the following ODBC drivers available for PyODBC:

SQL Server
SQL Server Native Client 11.0
Oracle in XE
SQL Server Native Client RDA 11.0
ODBC Driver 13 for SQL Server


In [3]:
# NOTE: YOUR ARGUMENTS WILL DEPEND ON YOUR SETUP. THIS WILL NOT WORK FOR YOU.
# Usually it's easier to use arguments
CONNECTION_ARGS = {
    'driver': '{SQL Server Native Client 11.0}',
    'server': os.environ['COMPUTERNAME'],
    'database': 'test',
    'trusted_connection': 'yes',
    'readonly': True,
}

# But if you manually want to specify your connection string, you can.
CONNECTION_STRING = '''DRIVER={SQL Server Native Client 11.0};
                       SERVER=kilo;
                       DATABASE=test;
                       Trusted_connection=yes'''

QUERY = '''

SELECT TOP(3)
    *
FROM
    TRACKS

'''

# Connection arguments allow us to open a connection like this:
conn = pyodbc.connect(**CONNECTION_ARGS)
# Then create a cursor.
cursor = conn.cursor()
# And run a query
cursor.execute(QUERY)
# And get results
rows = cursor.fetchmany(50)

# conn.commit()   # only if you need to commit a transaction
# conn.rollback() # only if you need to rollback a transaction
# Then we close our connection.
conn.close()

# Let's examine our results!
rows

[(1, 'Will Saul', 'Fastlane (Fink Remix)', datetime.date(2003, 3, 10)),
 (5, 'Louis Vega & Jay Sinister Sealee', 'Diamond Life (Bootleg Disco Remix)', datetime.date(2008, 9, 1)),
 (6, 'Aquarian Dream', 'Love and Tears (Soul Creatures Reworked Live Dub)', datetime.date(1999, 1, 1))]

### Again, this is way, way easier with a context manager (which automatically commits and closes):

In [4]:
# Connecting with a string is also possible.
with pyodbc.connect(CONNECTION_STRING) as conn:
    cursor = conn.cursor()
    cursor.execute(QUERY)
    data = cursor.fetchall()
    
# This handy idiom will give you back a dict instead.
# https://stackoverflow.com/questions/16519385/output-pyodbc-cursor-results-as-python-dictionary
columns = [column[0] for column in cursor.description]
result_list = [
    dict(zip(columns, row))
    for row
    in data
]
        
# Print our row-dict-a-ma-jigs.
for result_dict in result_list:
    print(result_dict)

{'id': 1, 'artist': 'Will Saul', 'track': 'Fastlane (Fink Remix)', 'date': datetime.date(2003, 3, 10)}
{'id': 5, 'artist': 'Louis Vega & Jay Sinister Sealee', 'track': 'Diamond Life (Bootleg Disco Remix)', 'date': datetime.date(2008, 9, 1)}
{'id': 6, 'artist': 'Aquarian Dream', 'track': 'Love and Tears (Soul Creatures Reworked Live Dub)', 'date': datetime.date(1999, 1, 1)}


### Notice above we didn't have to enter our password because it used our active directory credentials with the "integrated_authentication=True" option. Oracle databases will other parameters such as a username, password, and other information like a DBQ in a TNSNAMES.ORA file.

Reminders:

* never store your password on your computer.
* the password is stored as an unencrypted string unless you encrypt it.

In [5]:
# Lets get our credentials from the environment and enter our password
username = os.environ['USERNAME']
password = getpass.getpass('Enter your Oracle password here:')

# Warning: getpass stores your password in cleartext in RAM.
# print(password) # Uncommented, this would print your password!

Enter your Oracle password here:········


In [6]:
# For convenience, we will skip the connection string.
CONNECTION_ARGS = {
    'driver': '{Oracle in XE}',
    'user': os.environ['USERNAME'],
    'pwd': password,
    'readonly': True,
}

# DAS QUERY
QUERY = '''

SELECT
    first,
    last,
    house,
    gpa
FROM
    hogwarts

'''

In [7]:
# The API is the same, and the datatypes should convert automatically.

# Open context manager
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    # Create cursor
    cursor = conn.cursor()
    # Execute query
    cursor.execute(QUERY)
    # Fetch a single row
    first_row = cursor.fetchone()
    # Fetch all rows
    remaining_rows = cursor.fetchall()
        
# Print fetched remaing rows
for row in remaining_rows:
    print(row)

('Hermione', 'Granger', 'Gryffindor', Decimal('98.22'))
('Charlie', 'Weasley', 'Gryffindor', Decimal('80.25'))
('Draco', 'Malfoy', 'Slytherin', Decimal('95.22'))
('Susan', 'Bones', 'Hufflepuff', Decimal('89.22'))


### If you're lazy like me, the absolute easist thing you can do is to load directly into a dataframe using the read_sql() method.

In [8]:
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    # read_sql() is the bee's knees.
    hogwarts_df = pd.read_sql(QUERY, conn)

hogwarts_df

,FIRST,LAST,HOUSE,GPA
0,Harry,Potter,Gryffindor,92.21
1,Hermione,Granger,Gryffindor,98.22
2,Charlie,Weasley,Gryffindor,80.25
3,Draco,Malfoy,Slytherin,95.22
4,Susan,Bones,Hufflepuff,89.22


### You also have access to most ODBC thing-a-ma-bobs.

In [9]:
# Lets print out the available tables
with pyodbc.connect(**CONNECTION_ARGS) as conn:
    cursor = conn.cursor()
    table_list = list(cursor.tables())
    first_table = table_list[0][2]
    column_list = list(cursor.columns(table=first_table))

print('For table {}:'.format(first_table))

for column in column_list:
    print('\tColumn {} is of type {}.'.format(column[3], column[5]))

For table APEX_ACTIVITY_LOG:
	Column TIME_STAMP is of type DATE.
	Column COMPONENT_TYPE is of type VARCHAR2.
	Column COMPONENT_NAME is of type VARCHAR2.
	Column COMPONENT_ATTRIBUTE is of type VARCHAR2.
	Column INFORMATION is of type VARCHAR2.
	Column ELAP is of type NUMBER.
	Column NUM_ROWS is of type NUMBER.
	Column USERID is of type VARCHAR2.
	Column IP_ADDRESS is of type VARCHAR2.
	Column USER_AGENT is of type VARCHAR2.
	Column FLOW_ID is of type NUMBER.
	Column STEP_ID is of type NUMBER.
	Column SESSION_ID is of type NUMBER.
	Column SQLERRM is of type VARCHAR2.
	Column SQLERRM_COMPONENT_TYPE is of type VARCHAR2.
	Column SQLERRM_COMPONENT_NAME is of type VARCHAR2.
	Column PAGE_MODE is of type VARCHAR2.
	Column APPLICATION_INFO is of type VARCHAR2.
	Column WORKSHEET_ID is of type NUMBER.
	Column IR_SEARCH is of type VARCHAR2.
	Column IR_REPORT_ID is of type NUMBER.
	Column WEBSHEET_ID is of type NUMBER.
	Column WEBPAGE_ID is of type NUMBER.
	Column DATAGRID_ID is of type NUMBER.
	Col

# SQLite

Sqlite3 is a small, efficient database that is everywhere. Odds are it is in your pocket on your smartphone right now. It is great for making a database on disk or in RAM.

Reminders:
* DO NOT STORE DATA THAT SHOULD BE ON A SERVER ON YOUR LOCAL MACHINE.

### You can create a database on your local file system simply by using the sqlite3.connect() command.

In [10]:
BEETROOT_INVENTORY = [(5, 'red'), (3, 'purple'), (2, 'white')]

# Create a database in memory with the ':memory:' option.
conn = sqlite3.connect(':memory:')

# Create a cursor
cursor = conn.cursor()

# Create a table and populate with values from our list above.
cursor.execute('CREATE TABLE rutabagas(key INTEGER, pounds INTEGER, color TEXT, PRIMARY KEY(key DESC));')
cursor.executemany('INSERT INTO rutabagas VALUES(NULL, ?, ?)', BEETROOT_INVENTORY)

# Commit changes to database.
conn.commit()

# Select everything in our database and print it out.
cursor.execute('SELECT * FROM rutabagas')
print(cursor.fetchall())

# Connection closes, and database disappears
conn.close()

[(1, 5, 'red'), (2, 3, 'purple'), (3, 2, 'white')]


### More importantly, we can also write the DB to disk

Note: Again, context managers are the best way to do this, because they commit and close automatically.

In [11]:
# Declare path to test database.
DB_PATH = './data/test.sqlite3' # often seen as test.db 

with sqlite3.connect(DB_PATH) as conn:
    # Create a cursor
    cursor = conn.cursor()
    # Create a table and populate with values from our list above.
    cursor.execute('CREATE TABLE IF NOT EXISTS rutabagas(key INTEGER, pounds INTEGER, color TEXT, PRIMARY KEY(key DESC));')
    cursor.executemany('INSERT INTO rutabagas VALUES(NULL, ?, ?)', BEETROOT_INVENTORY)
    # Implicit commit and close.

### As usual, things are generally better with pandas ...

In [12]:
# Read our CSV into a dataframe
df = pd.read_csv('./data/iris_dataset.csv')

# Open SQLite connection to write
with sqlite3.connect('./data/test.sqlite3') as conn:
    # Write to database, which allows you to add lots of files to a single source.
    df.to_sql('iris_table', conn, if_exists='replace', index=False)

In [13]:
# Read to prove we wrote it
with sqlite3.connect('./data/test.sqlite3') as conn:
    # Same as before.
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM iris_table LIMIT 5;')
    records = cursor.fetchall()

records

[(5.1, 3.5, 1.4, 0.2, 'Iris-setosa'),
 (4.9, 3.0, 1.4, 0.2, 'Iris-setosa'),
 (4.7, 3.2, 1.3, 0.2, 'Iris-setosa'),
 (4.6, 3.1, 1.5, 0.2, 'Iris-setosa'),
 (5.0, 3.6, 1.4, 0.2, 'Iris-setosa')]

# Additional Learing Resources

* ### [SQLite3.org](https://www.sqlite.org/index.html)


---

# Next Up: [Files](4_files.ipynb)


<img style="margin-left: 0;" src="static/file.png" width="100">

---